In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

file_name = '/mnt/raw_data/01_tiff_stacks/2022-12-15/patch_voltage-001/patch_voltage-001_Cycle00001_VoltageRecording_001.csv'
recording = pd.read_table(file_name, sep=r',', skipinitialspace=True)
print(recording.head())
# calculate the median value of the Command column
median = recording['Command'].median()

# subtract the median from each value in the Command column
recording['Command'] = recording['Command'] - median
xml_file_name = '/mnt/raw_data/00_raw_data/2022-12-15/patch_voltage-008/patch_voltage-008_Cycle00001_VoltageRecording_001.xml'
tree = ET.parse(xml_file_name)
root = tree.getroot()
# create a dictionary of dictionaries for each VRecSignal node
signals_dict = {}
for signal in root.findall(".//VRecSignal"):
    name = signal.find("Name").text
    unit_name = signal.find("Unit/UnitName").text
    unit_multiplier = float(signal.find("Unit/Multiplier").text)
    unit_divisor = float(signal.find("Unit/Divisor").text)
    signals_dict[name] = {
        "unit_name": unit_name,
        "unit_multiplier": unit_multiplier,
        "unit_divisor": unit_divisor
    }
# access the values for a specific key
in0_unit_name = signals_dict["IN_0"]["unit_name"]
in0_unit_multiplier = signals_dict["IN_0"]["unit_multiplier"]
in0_unit_divisor = signals_dict["IN_0"]["unit_divisor"]
command_unit_name = signals_dict["Command"]["unit_name"]
command_unit_multiplier = signals_dict["Command"]["unit_multiplier"]
command_unit_divisor = signals_dict["Command"]["unit_divisor"]
print(signals_dict)


fig, axs = plt.subplots(1,2, figsize=(12,5))

axs[0].plot(recording['Time(ms)'], recording['IN_0']/in0_unit_divisor)
axs[0].set_xlabel('Time (ms)')
axs[0].set_ylabel(in0_unit_name)
axs[0].set_title('IN_0')
#axs[0].set_ylim(-80,200)
axs[1].plot(recording['Time(ms)'], recording['Command']/command_unit_divisor)
axs[1].set_xlabel('Time (ms)')
axs[1].set_ylabel(command_unit_name)
axs[1].set_title('Command')
#axs[1].set_ylim(0,500)
fig.tight_layout()

ModuleNotFoundError: No module named 'matplotlib'

In [70]:
xml_protocol_file_name = '//mnt/raw_data/00_raw_data/2022-12-15/patch_voltage-008/patch_voltage-008_Cycle00001_VoltageOutput_001.xml'
tree = ET.parse(xml_protocol_file_name)
root = tree.getroot()
# Get the names of the waveforms used in the recording
waveform_names = {}

for waveform in root.findall("Waveform//"): # // Selects all subelements, on all levels beneath the current element
    for name in waveform.findall("Name"):
        waveform_names[name] = name.text
waveform_names = list(waveform_names.values())

waveform_enabled = {}
for waveform in root.findall("Waveform//"):
    for enabled in waveform.findall("Enabled//"):
        waveform_enabled[enabled] = enabled.text
waveform_enabled = list(waveform_enabled.values())

# Get the channel names in the recording
channel_names = {}
for channel in root.findall("Waveform"):
    for name in channel.findall("Name"):
        channel_names[name] = name.text
channel_names = list(channel_names.values())


# Get the units of each channel
channel_units = {}
for channel in root.findall("Waveform"):
    for unit in channel.findall("Units"):
        channel_units[unit] = unit.text
channel_units = list(channel_units.values())

# Get the channels that are actually in use
channel_in_use = {}
for channel in root.findall("Waveform"):
    for usage in channel.findall("Enabled"):
        channel_in_use[usage] = usage.text
channel_in_use = list(channel_in_use.values())




def _element_to_dict(element):
    result = {}
    for child in element:
        if len(child) == 0:
            result[child.tag] = child.text
        else:
            result[child.tag] = _element_to_dict(child)
    return result if len(result) > 0 else None

def parse_xml_file(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    data_list = []
    for element in root:
        data_dict = _element_to_dict(element)
        if data_dict:
            data_list.append(data_dict)
    return data_list

data_list = parse_xml_file(xml_protocol_file_name)

enabled_list = []
for data_dict in data_list:
    if data_dict.get("Enabled") == "true":
        enabled_list.append(data_dict.get('Name'))
print("Active channels in recording: %s" % enabled_list)

Active channels in recording: ['Command', 'IN 0']


In [73]:
inner_value = data_list[0]['0']['WaveformComponent']
print(inner_value)

KeyError: '0'

In [87]:
def enumerate_channels(channel_list):
    """
    Separate each dictionary entry in a list into a new variable called channel_x where x is the index in the list.
    :param channel_list: a list of dictionaries
    :return: a dictionary with keys channel_1, channel_2, channel_3, etc.
    """
    channel_dict = {}
    for i, channel in enumerate(channel_list):
        name = channel['Name']
        channel_dict['channel_{}'.format(i)] = channel
    return channel_dict
channels = enumerate_channels(data_list)
print(channels['channel_0']['WaveformComponent_PulseTrain'])

{'Name': 'cc_steps', 'PulseCount': '5', 'PulseWidth': '250', 'PulseSpacing': '250', 'PulsePotentialStart': '0.25', 'PulsePotentialDelta': '0.25', 'RestPotential': '0', 'FirstPulseDelay': '250', 'Repetitions': '3', 'DelayBetweenReps': '2000', 'isLowTimeChecked': 'true'}


In [93]:
def find_waveform_components(data_dict):
    component_dict = {}
    for dict_name, data in data_dict.items():
        components = []
        for key, value in data.items():
            if isinstance(value, dict) and key.startswith('WaveformComponent_'):
                components.append(key)
        component_dict[dict_name] = components
    return component_dict
find_waveform_components(channels)

{'channel_0': ['WaveformComponent_PulseTrain'],
 'channel_1': [],
 'channel_2': ['WaveformComponent_PatchAmplifier'],
 'channel_3': ['WaveformComponent_PulseTrain'],
 'channel_4': [],
 'channel_5': [],
 'channel_6': [],
 'channel_7': []}

In [117]:
def parse_voltage_recording_xml(test_xml_file_name):
    _tree = ET.parse(test_xml_file_name)
    _root = _tree.getroot()
    # create a dictionary of dictionaries for each VRecSignal node
    _signals_dict = {}
    rec_name = []
    date_name = []
    for signal in _root.findall(".//VRecSignal"):
        name = signal.find("Name").text
        unit_name = signal.find("Unit/UnitName").text
        unit_multiplier = float(signal.find("Unit/Multiplier").text)
        unit_divisor = float(signal.find("Unit/Divisor").text)
        try:
            unit_device = signal.find("Unit/PatchClampDevice").text
        except:
            pass
        try:
            unit_channel = signal.find("Unit/PatchClampChannel").text
        except:
            pass
        signal_type = signal.find("Type").text
        signal_gain = float(signal.find("Gain").text)
        signal_channel_num = int(signal.find("Channel").text)
        signal_enabled = signal.find("Enabled").text

        _signals_dict[name] = {
            "unit_name": unit_name,
            "unit_multiplier": unit_multiplier,
            "unit_divisor": unit_divisor
        }
    acquisition_time_ms = _root.find("Experiment/AcquisitionTime").text
    sampling_rate = _root.find("Experiment/Rate").text
    _recording_name = _root.find("DataFile").text
    _recording_datetime = _root.find("DateTime").text
    rec_name.append(_recording_name)
    date_name.append(_recording_datetime)


    return _signals_dict, rec_name, date_name, acquisition_time_ms


voltage_rec_xml = '/mnt/raw_data/00_raw_data/2022-12-15/patch_voltage-008/patch_voltage-008_Cycle00001_VoltageRecording_001.xml'
signal_list,recording_name, recording_datetime, acquisition_time = parse_voltage_recording_xml(voltage_rec_xml)
print(signal_list)
print(recording_name)
print(recording_datetime)
print(acquisition_time)

{'IN_0': {'unit_name': 'mV', 'unit_multiplier': 1.0, 'unit_divisor': 0.01}, 'Command': {'unit_name': 'pA', 'unit_multiplier': 1.0, 'unit_divisor': 0.0005}, 'Input 2': {'unit_name': 'V', 'unit_multiplier': 1.0, 'unit_divisor': 0.001}, 'Input 3': {'unit_name': 'Volt', 'unit_multiplier': 1.0, 'unit_divisor': 1.0}, 'LED': {'unit_name': 'Volt', 'unit_multiplier': 1.0, 'unit_divisor': 1.0}, 'Locomotion': {'unit_name': 'Volt', 'unit_multiplier': 1.0, 'unit_divisor': 1.0}, 'Input 6': {'unit_name': 'Volt', 'unit_multiplier': 1.0, 'unit_divisor': 1.0}, 'Input 7': {'unit_name': 'Volt', 'unit_multiplier': 1.0, 'unit_divisor': 1.0}}
['patch_voltage-008_Cycle00001_VoltageRecording_001']
['2022-12-15T15:00:18.144625-05:00']
20000


In [96]:
def enumerate_signals(signal_list):
    """
    Separate each dictionary entry in a list into a new variable called channel_x where x is the index in the list.
    :param channel_list: a list of dictionaries
    :return: a dictionary with keys channel_1, channel_2, channel_3, etc.
    """
    signal_dict = {}
    for i, signal in enumerate(signal_list):
        name = channel['Name']
        signal_dict['signal_{}'.format(i)] = signal
    return signal_dict
signals = enumerate_signals(signal_list)

In [113]:
print(recording['Time(ms)'].max)

<bound method NDFrame._add_numeric_operations.<locals>.max of 0             0.0
1             0.1
2             0.2
3             0.3
4             0.4
           ...   
199996    19999.6
199997    19999.7
199998    19999.8
199999    19999.9
200000    20000.0
Name: Time(ms), Length: 200001, dtype: float64>


In [1]:
import matplotlib.pyplot as plt
file_name = '/mnt/raw_data/01_tiff_stacks/2022-12-15/patch_voltage-003/patch_voltage-003_Cycle00001_VoltageRecording_001.csv'
recording = pd.read_table(file_name, sep=r',', skipinitialspace=True)
fig, axs = plt.subplots(1,2, figsize=(12,5))

axs[0].plot(recording['Time(ms)'], recording['IN_0']/0.005)
axs[0].set_xlabel('Time (ms)')
axs[0].set_ylabel(in0_unit_name)
axs[0].set_title('IN_0')

ModuleNotFoundError: No module named 'matplotlib'